**Some relevant Tensorflow and Keras modules that we will be using for our LSTM model:**

- https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding 
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM
- https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical
- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/TimeDistributed
- https://www.tensorflow.org/api_docs/python/tf/data/Dataset


**Keras modules required to train the feed-forward model**


In [ ]:

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, LSTM
import tensorflow as tf

dummy_input_size = 5
dummy_vocab_size = 100
dummy_emb_size=20
dummy_nlabels=17
dummy_hidden_dim=128

model = Sequential()
model.add(Input(shape=(dummy_input_size,),dtype=tf.int32)) 

model.add(Embedding(input_dim=dummy_vocab_size, output_dim=dummy_emb_size,
                    mask_zero=True, 
                    input_length=dummy_input_size))

model.add(Flatten())

model.add(Dense(dummy_hidden_dim, activation='relu'))
model.add(Dense(dummy_nlabels, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 20)             2000      
                                                                 
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               12928     
                                                                 
 dense_2 (Dense)             (None, 17)                2193      
                                                                 
Total params: 17,121
Trainable params: 17,121
Non-trainable params: 0
_________________________________________________________________


**Keras modules required to create our LSTM model**




In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences


dummy_num_words = 100
dummy_nlabels = 50
dummy_emb_dim = 20
dummy_max_seq_length = 64
lstm_out_dim = 128

model = Sequential()
model.add(Embedding(dummy_num_words, dummy_emb_dim, input_length=dummy_max_seq_length, mask_zero=True))
model.add(LSTM(lstm_out_dim, return_sequences=True))
model.add(TimeDistributed(Dense(dummy_nlabels, activation='softmax')))

model.summary()



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 64, 20)            2000      
                                                                 
 lstm_6 (LSTM)               (None, 64, 128)           76288     
                                                                 
 time_distributed_5 (TimeDis  (None, 64, 50)           6450      
 tributed)                                                       
                                                                 
Total params: 84,738
Trainable params: 84,738
Non-trainable params: 0
_________________________________________________________________


**We can save the model after training is finished** (ignore the WARNING for this specific example)

In [ ]:
model.save("dummy_model.h5")

**We also can load the model to use it anytime later**

In [ ]:
import tensorflow
model = tensorflow.keras.models.load_model("dummy_model.h5")

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 64, 20)            2000      
                                                                 
 lstm_6 (LSTM)               (None, 64, 128)           76288     
                                                                 
 time_distributed_5 (TimeDis  (None, 64, 50)           6450      
 tributed)                                                       
                                                                 
Total params: 84,738
Trainable params: 84,738
Non-trainable params: 0
_________________________________________________________________


**pad_sequences: pads contexts that have less than max_seq_length words. For instance:**

In [ ]:
sequences = [[],[1],[3,2],[4,5,7,8]]
pad_sequences(sequences)

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 3, 2],
       [4, 5, 7, 8]], dtype=int32)

**to_categorical: transforms a set of discrete classes into their one-hot representation**


In [ ]:
import tensorflow
discrete_classes = [0, 1, 2, 3, 0, 4, 5, 3, 3, 3]
print ("Discrete classes:", discrete_classes)
categorical_classes = tensorflow.keras.utils.to_categorical(discrete_classes, num_classes=len(set(discrete_classes)))
print ("Categorical classes:")
print (categorical_classes)

Discrete classes: [0, 1, 2, 3, 0, 4, 5, 3, 3, 3]
Categorical classes:
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


**Create a set of samples using [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) and use it to train any keras model**

In [25]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Dummy training data, three possible inputs and their corresponding outputs
X = [[0,0,1],[0,1,0],[1,0,0]]
y = [0,0,1]

weights = [0.5,0.5, 1.25]

dummy_batch_size=1
use_weights=False

if use_weights:
  dataset = tf.data.Dataset.from_tensor_slices((X, y, weights))
  dataset = dataset.batch(dummy_batch_size)
else:
  dataset = tf.data.Dataset.from_tensor_slices((X, y))
  dataset = dataset.batch(dummy_batch_size)

# Define the logistic regression model
model = Sequential()
model.add(Dense(1, input_dim=3, activation='sigmoid'))

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], weighted_metrics=[])

# Train the model on the input and output data
model.fit(dataset, epochs=1, verbose=1)

# Evaluate the model on the same input data (here we do no respect the train/dev methodology, but it's just for illustrative purposes)
loss, accuracy = model.evaluate(dataset, verbose=0)

print ("Final loss/accuracy", loss, accuracy)

3/3 [==============================] - 0s 5ms/step - loss: 0.6434 - accuracy: 0.6667
Final loss/accuracy 0.6423259377479553 0.6666666865348816
